In [3]:
import nltk

In [4]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

a='Whats would you do with 1000000$ ? '

b='organizer Organizing Organization'

tokenize(a)

['Whats', 'would', 'you', 'do', 'with', '1000000', '$', '?']

In [5]:
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

def stem(word):
    return stemmer.stem(word.lower())

stem_words = [stem(word) for word in tokenize(b)]

stem_words

['organ', 'organ', 'organ']

In [6]:
import numpy as np

def bag_of_words(tokenized_sentence,all_words):
    stemmed_sentence=[stem(w) for w in tokenized_sentence]
    bag=[0]*len(all_words)
    for index,word in enumerate(all_words):
        if word in stemmed_sentence:
            bag[index]=1
    return bag


In [7]:
import json
with open('intent.json','r') as f:
    data=json.load(f)

In [9]:
tags=[]
responses=[]
all_words=[]
xy=[]
for intent in data['intents']:
    tag=intent['tag']
    if tag not in tags:
        tags.append(tag)
    for pattern in intent['patterns']:
        w=tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag))
    for response in intent['responses']:
        responses.append(response)


In [20]:
ignore_words=['!','?','.',',']
all_words=[stem(w) for w in all_words if w not in ignore_words]
all_words=list(set(sorted(all_words)))
tags=list(set(sorted(tags)))
tags

['programming_language',
 'Company_details',
 'outcome',
 'thanks',
 'course_enroll',
 'specific_questions2',
 'suggestions',
 'prerequisite',
 'payment_platforms',
 'liveclassesprice',
 'size',
 'difference',
 'special',
 'classes_levels',
 'statistics',
 'specific_questions',
 'offers',
 'monitor',
 'courses',
 'mentors',
 'course_levels',
 'liveclassescontents',
 'help',
 'mulitple_courses',
 'achievements',
 'eligible',
 'partners',
 'coding_language',
 'specific_questions5',
 'noanswer',
 'goodbye',
 'greeting',
 'specific_questions4',
 'missed_class',
 'time',
 'contact',
 'specific_questions3',
 'specific_questions1',
 'change',
 'coding',
 'coursecontents',
 'class_enroll',
 'options',
 'cancelations-refunds',
 'values',
 'courseprice']

In [21]:
import random
training=[]
for tokenized_sentence,tag in xy:
    bag=bag_of_words(tokenized_sentence,all_words)
    output=[0]*len(tags)
    output[tags.index(tag)]=1
    training.append([bag,output])
random.shuffle(training)
training=np.array(training)


<ipython-input-21-08c09ac4ef71>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training=np.array(training)


In [12]:
X_train=list(training[:,0])
y_train=list(training[:,1])

In [13]:
import pickle
pickle.dump({'all_words':all_words, 'xy':xy, 'responses': responses,'X_train':X_train, 'y_train':y_train},open("training_data","wb"))

In [18]:
with open('training_data','rb') as f:
    data = pickle.load(f)
all_words = data['all_words']
xy = data['xy']
responses = data['responses']
X_train = data['X_train']
y_train = data['y_train']

In [11]:
import tensorflow as tf
import tflearn


input_layer=tflearn.input_data(shape=[None,len(X_train[0])])
layer1=tflearn.fully_connected(input_layer,10)
layer2=tflearn.fully_connected(layer1,10)
output_layer=tflearn.fully_connected(layer2,len(y_train[0]),activation='softmax')
net=tflearn.regression(output_layer,optimizer='adam',loss='categorical_crossentropy',metric=tflearn.metrics.Accuracy())
model=tflearn.DNN(net, tensorboard_dir='tflearn_logs')

model.fit(X_train, y_train, n_epoch=1000, batch_size=8, show_metric=True)


Training Step: 30999  | total loss: 0.04361 | time: 0.103s
| Adam | epoch: 1000 | loss: 0.04361 - acc: 0.9555 -- iter: 240/244
Training Step: 31000  | total loss: 0.03925 | time: 0.107s
| Adam | epoch: 1000 | loss: 0.03925 - acc: 0.9599 -- iter: 244/244
--


In [12]:
def bow(sentence,all_words):
    stemmed_sentence=[stem(w) for w in tokenize(sentence)]
    bag=[0]*len(all_words)
    for index,word in enumerate(all_words):
        if word in stemmed_sentence:
            bag[index]=1
    return np.array(bag)

In [13]:
model.predict([bow('Is anyone there?', all_words)])

array([[2.41278791e-16, 7.38160935e-33, 5.91688157e-11, 9.99958396e-01,
        1.56103013e-25, 1.21264052e-35, 4.96906103e-28, 1.67443126e-21,
        2.22879726e-27, 6.49067624e-20, 1.30199143e-27, 2.43885239e-26,
        2.31730271e-32, 1.50878557e-37, 8.61871392e-13, 1.73337968e-28,
        0.00000000e+00, 9.87313069e-06, 2.00278405e-23, 1.83506802e-06,
        4.73738300e-23, 1.50529325e-33, 1.11494306e-22, 7.21014049e-20,
        2.12551452e-29, 4.63704385e-30, 6.79855919e-29, 1.34679005e-34,
        7.03866908e-06, 2.92133791e-06, 1.65516501e-19, 4.27909661e-14,
        1.73657241e-10, 1.38534970e-08, 1.62432746e-20, 2.46202825e-31,
        8.78697805e-24, 1.57747818e-05, 2.27391227e-25, 1.11552285e-20,
        8.76535064e-24, 2.02862839e-22, 2.93931239e-21, 7.31819888e-23,
        4.62357835e-25, 4.15373734e-06]], dtype=float32)

In [14]:
ERROR_THRESHOLD = 0.70
def classify(sentence):
    #generate probabilities from model
    results = model.predict([bow(sentence, all_words)])[0]
    
    # filter out predictions below a threashold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    
    # sort by strength of the probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((tags[r[0]], r[1]))
    return return_list

In [15]:
classify('bye')

[('goodbye', 0.999992)]

In [41]:

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in data['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))

In [43]:
bot_name = "Bot"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == 'quit':
        break
    if response(sentence):
        "Sam :"+response(sentence)

Let's chat! (type 'quit' to exit)
You: 'What are the ways to enroll to the course?',
Choose Course which you want to enroll yourself.Fill the details Make your payment and enjoy CODING
You: quit
